In [1]:
import os
os.chdir('/Users/jenif/Documents/My Data Science Project/Exploratory Analytics of Crop Production Trend Across Malaysia/Dataset')
os.getcwd()

'C:\\Users\\jenif\\Documents\\My Data Science Project\\Exploratory Analytics of Crop Production Trend Across Malaysia\\Dataset'

**1. Import Packages**

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


**2. Data Load: Crop Dataset**

In [5]:
import numpy as np
import pandas as pd

mdf=pd.read_csv("final_crop.csv")

In [6]:
mdf.head()

,state,date,crop_type,planted_area,production,year
0,Johor,2017-01-01,cash_crops,3780.3,42424.1,2017
1,Johor,2018-01-01,cash_crops,3483.7,43420.4,2018
2,Johor,2019-01-01,cash_crops,3007.5,38566.7,2019
3,Johor,2020-01-01,cash_crops,3056.2,42963.0,2020
4,Johor,2021-01-01,cash_crops,3522.4,47538.7,2021


In [7]:
model_df= mdf.copy()

**3. Define feature and target**

In [9]:
# Features and target
X = model_df[['state', 'crop_type', 'year']]
y = model_df['production']
y = np.log1p(y)

**4. Encoding**

In [11]:
categorical_features = ['state', 'crop_type']
numeric_features = ['year']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ]
)

**Train-test-split data**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


**Model Development and Evaluation**

In [15]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42)
}


In [32]:
results = []

results = []

for name, model in models.items():

    # Create full pipeline (preprocess + model)
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Train
    pipeline.fit(X_train, y_train)

    # Predict
    y_pred = pipeline.predict(X_test)

    # If using log transform, reverse it
    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred)

    # Evaluate
    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
    r2 = r2_score(y_test_actual, y_pred_actual)

    results.append({
        "Model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2 Score": r2
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

print(results_df.sort_values(by="RMSE"))


best_model = None
best_rmse = float("inf")
best_name = ""

for name, model in models.items():

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)

    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred)

    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))

    if rmse < best_rmse:
        best_rmse = rmse
        best_model = pipeline
        best_name = name

print(f"Best Model: {best_name}")


               Model           MAE          RMSE  R2 Score
1      Random Forest  2.631130e+05  1.806702e+06  0.980428
2            XGBoost  2.499308e+05  1.926019e+06  0.977758
0  Linear Regression  1.584165e+06  9.891955e+06  0.413294
Best Model: Random Forest


Tree-based ensemble models significantly outperformed Linear Regression, indicating strong non-linear relationships between crop type, state, and production levels.

Random Forest achieved the highest R² (0.98) and lowest RMSE, suggesting superior generalization and better handling of high-production states.

Therefore, Random Forest was selected as the final production model

**Save Model**

In [36]:
import joblib

joblib.dump(best_model, "best_model.pkl")


['best_model.pkl']